In [3]:
import os
os.environ['HF_HOME'] = '/workspace/huggingface'

from transformer_lens import HookedTransformer, ActivationCache, utils
import torch

if torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Device: {device}")

import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from functools import partial
tqdm.pandas()

init_notebook_mode(connected=True)

Device: mps


Matplotlib is building the font cache; this may take a moment.



## Data Loading

In [5]:
%%bash
git clone https://github.com/asaparov/prontoqa.git
cd prontoqa
mkdir json
unzip generated_ood_data.zip -d json

Cloning into 'prontoqa'...


Archive:  generated_ood_data.zip
  inflating: json/1hop_AndElim_random_noadj.json  
  inflating: json/1hop_AndIntro_random_noadj.json  
  inflating: json/1hop_OrElim_3proofwidth_testrandom_nodistractor_testdistractor_noadj.json  
  inflating: json/1hop_ProofByContra_testrandom_nodistractor_testdistractor_noadj.json  
  inflating: json/1hop_ProofsOnly_random_noadj.json  
  inflating: json/2hop_AndElim_testrandom_nodistractor_testdistractor_noadj.json  
  inflating: json/2hop_AndIntro_testrandom_nodistractor_testdistractor_noadj.json  
  inflating: json/2hop_OrIntro_random_noadj.json  
  inflating: json/2hop_OrIntro_testrandom_nodistractor_testdistractor_noadj.json  
  inflating: json/2hop_ProofsOnly_testrandom_nodistractor_testdistractor_noadj.json  
  inflating: json/1hop_AndElim_2testhops_random_noadj.json  
  inflating: json/1hop_AndElim_3testhops_random_noadj.json  
  inflating: json/1hop_AndIntro_2testhops_random_noadj.json  
  inflating: json/1hop_AndIntro_3testhops_random_noadj.j

In [6]:
import json

def load_json(file):
    with open(file, 'r') as f:
        return json.load(f)
    
data = load_json("prontoqa/json/1hop_ProofsOnly_random_noadj.json")

In [17]:
from random import randint

randint(0, 1)

1

In [40]:
from random import randint

n_shots = 3
labels = ["is", "is not"]

prompt = ""

for i in range(n_shots):
    change = randint(0, 1)
    prompt += data['example1'][f"in_context_example{i}"]['question'] + "\n"
    query = data['example1'][f"in_context_example{i}"]['query']
    
    if "is not" in query:
        label = "True" if change == 1 else "False"
        query = query.replace("is not", labels[change])
    else:
        label = "True" if change == 0 else "False"
        query = query.replace("is", labels[change])

    prompt += query.replace("Prove: ", "True or False: ") + "\n\n"

    for j, step in enumerate(data['example1'][f"in_context_example{i}"]['chain_of_thought']):
        prompt += f"({j+1}) {step}\n"
    
    prompt += f"\nAnswer: {label}\n\n"

change = randint(0, 1)
prompt += data['example1']["test_example"]['question'] + "\n"
query = data['example1']["test_example"]['query']

if "is not" in query:
    label = "True" if change == 1 else "False"
    query = query.replace("is not", labels[change])
else:
    label = "True" if change == 0 else "False"
    query = query.replace("is", labels[change])

prompt += query.replace("Prove: ", "True or False: ") + "\n\n"
    
print(prompt[-500:], label)

Each vumpus is a brimpus. Fae is a sterpus. Fae is a vumpus.
True or False: Fae is not slow.

(1) Fae is a vumpus.
(2) Each vumpus is not slow.
(3) Fae is not slow.

Answer: True

Gorpuses are sterpuses. Gorpuses are grimpuses. Every dumpus is a jompus. Every grimpus is a shumpus. Gorpuses are not small. Sterpuses are liquid. Every shumpus is not muffled. Dumpuses are bright. Each grimpus is a brimpus. Every grimpus is not cold. Wren is a dumpus. Wren is a grimpus.
True or False: Wren is cold.

 False
